In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
libs = ['SRR1197125']
path_libs = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/sun_deg__%s'
path_transcripts = '/home/juan/Desktop/juan/bio/mirna_mite/data/fixed.cdna.all.fa'
path_transcripts_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/targets_mites.fa'
path_blast_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/transcripts_mites.csv'
path_mirna_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results.csv'
path_res_final = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/res_deg.csv'

In [ ]:
dfs = []
for lib in libs:
    path_lib = path_libs % lib
    df = pd.read_csv(path_lib, sep='\t',comment='#')
    df['lib'] = lib
    dfs.append(df)

In [ ]:
df = pd.concat(dfs)

In [ ]:
df.head(2)

In [ ]:
transcripts = set()
for k,v in df.iterrows():
    transcript = v.Transcript
    transcripts.add(transcript)

In [ ]:
fasta_seq = SeqIO.parse(path_transcripts, 'fasta')


In [ ]:
buffer_seqs = []
for record in fasta_seq:
    if record.id in transcripts:
        buffer_seqs.append(record)
SeqIO.write(buffer_seqs, path_transcripts_res, "fasta")
print(path_transcripts_res)

In [4]:
df_mites = pd.read_csv(path_blast_res, sep="\t")
cols = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs','score']
df_mites.columns = cols
df_mites = df_mites[df_mites.pident >= 85]
df_mites = df_mites[df_mites.qcovs >= 85]
print(len(df_mites.index))
df_mites.head(2)

3223


,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
0,MITE_0,TraesCS5D02G132100.1,1,80,479,399,6,1,91.358,2.370000e-26,81,80,2089,100,122
1,MITE_0,TraesCS5B02G114400.1,1,80,2097,2177,6,1,91.358,2.370000e-26,81,80,2243,100,122


In [5]:
df_mites[df_mites.sseqid=='TraesCS2A02G580000.2']

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
13,MITE_0,TraesCS2A02G580000.2,1,80,2325,2405,7,1,90.123,2.890000e-25,81,80,2747,100,118
14,MITE_0,TraesCS2A02G580000.2,1,80,2405,2325,7,1,90.123,2.890000e-25,81,80,2747,100,118
1743,MITE_187,TraesCS2A02G580000.2,5,55,2390,2340,7,0,86.275,2.510000e-11,51,59,2747,86,66
2755,DTT_Null_Minimus_DQ885757-1,TraesCS2A02G580000.2,1,30,2323,2352,2,0,93.333,4.980000e-07,30,55,2747,100,50
2756,DTT_Null_Minimus_DQ885757-1,TraesCS2A02G580000.2,26,55,2353,2323,3,1,87.097,1.000000e-03,31,55,2747,100,38
3025,MITE_296,TraesCS2A02G580000.2,2,90,2410,2322,12,0,86.517,3.570000e-25,89,94,2747,95,118
3989,MITE_390,TraesCS2A02G580000.2,1,85,2323,2407,10,0,88.235,8.980000e-26,85,85,2747,100,120
3990,MITE_390,TraesCS2A02G580000.2,1,85,2407,2323,12,0,85.882,4.650000e-23,85,85,2747,100,110
4369,DTT_Taes_Athos_BJ268809-1,TraesCS2A02G580000.2,1,66,2405,2340,9,0,86.364,2.060000e-16,66,108,2747,94,86
4371,DTT_Taes_Athos_BJ268809-1,TraesCS2A02G580000.2,73,107,2360,2326,1,0,97.143,1.930000e-10,35,108,2747,94,64


In [6]:
df['new_start'] = df[['TStart','TStop']].min(axis=1)
df['new_end'] = df[['TStart','TStop']].max(axis=1)
df['TStart'] = df['new_start']
df['TStop'] = df['new_end']
df = df.drop('new_start',axis=1).drop('new_end',axis=1)


NameError: name 'df' is not defined

In [224]:
df_mites['new_start'] = df_mites[['sstart','send']].min(axis=1)
df_mites['new_end'] = df_mites[['sstart','send']].max(axis=1)
df_mites['sstart'] = df_mites['new_start']
df_mites['send'] = df_mites['new_end']
df_mites = df_mites.drop('new_start',axis=1).drop('new_end',axis=1)


In [225]:
df['MITE'] = ''

In [226]:
for k,v in df.iterrows():
    start = v.TStart
    end = v.TStop
    transcript = v.Transcript
    df_filter = df_mites[df_mites.sseqid==transcript]
    df_filter = df_filter[(df_filter.sstart <= start) & (df_filter.send >= end)]
    if len(df_filter.index) > 0:
        mites = ','.join(df_filter.qseqid.tolist())
        df.at[k, 'MITE'] += mites
        

In [227]:
total = len(df.index)
w_mites = len(df[df.MITE != ''].index)
w_o_mites = len(df[df.MITE == ''].index)
print('Total', total)
print('W mites', w_mites)
print('WO mites', w_o_mites)
print(w_mites * 100 / total)

Total 619
W mites 126
WO mites 493
20.355411954765753


In [ ]:
df_merge.to_csv(path_res_final, index=None)
path_res_final

In [228]:
df_mites = df[df.MITE != '']
df[df.MITE != ''][['SiteID','Query','Transcript','TStart','TStop','MITE','Sequence']].head()

,SiteID,Query,Transcript,TStart,TStop,MITE,Sequence
4,TraesCS1A02G116200.1:3849,mrcv_all_Cluster_12767,TraesCS1A02G116200.1,3838,3858,"MITE_803,MITE_803",UACUCCCUCCGUUCCAAAAUA&UAAUAUGGAUUGGAGGGGGUA
5,TraesCS1A02G116200.2:3785,mrcv_all_Cluster_12767,TraesCS1A02G116200.2,3774,3794,"MITE_803,MITE_803",UACUCCCUCCGUUCCAAAAUA&UAAUAUGGAUUGGAGGGGGUA
7,TraesCS1A02G138200.2:1948,sun_all_Cluster_8835,TraesCS1A02G138200.2,1936,1956,DTT_Taes_Athos_BJ319864-1,UGUCUUACAUUAUG-GGACAGA&UCUGUUCACA-AAUGUAAGACG
9,TraesCS1A02G143400.1:16299,mrcv_mites_MITE_T_120271,TraesCS1A02G143400.1,16288,16308,"MITE_96,MITE_96,MITE_148,MITE_148,MITE_179,MIT...",CUCCCUCCGUUCCGAAUUACU&UAUAUUUUGGUACGGAGGGAU
10,TraesCS1A02G143400.1:16305,mrcv_mites_MITE_T_100589,TraesCS1A02G143400.1,16294,16314,"MITE_96,MITE_96,MITE_148,MITE_148,MITE_179,MIT...",CCGUUCCGAAUUACUUGUCUU&AUGACGAGUAAAUCAGAACGG


In [229]:
df_mirna_res = pd.read_csv(path_mirna_res, sep='\t')
df_mirna_res.rename(columns={'MITE':'MITE_prod'}, inplace=True)
print(len(df_mirna_res.index))
df_mirna_res.head(2)

260


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,22,23,24,seqname,position,start,end,MITE_prod,mirbase,annotation
0,1A:309399489-309399664,mrcv_all_Cluster_872,176,553,4.340,1,0.002,-,ACGGCAUAGAGGCACUGCAAA,327,...,61,5,4,1A,309399489-309399664,309399489,309399664,NaN,NaN,intergenic
1,1A:309399491-309399581,sun_all_Cluster_2789,91,459,3.635,4,0.007,-,ACGGCAUAGAGGCACUGCAAA,196,...,26,0,5,1A,309399491-309399581,309399491,309399581,NaN,NaN,intergenic


In [230]:
df_mirna_res_mites = df_mirna_res[df_mirna_res.MITE_prod.notnull()]
print(len(df_mirna_res_mites.index))


28


In [231]:
df_mirna_res_mites['Name'] = df_mirna_res_mites.Name.str.split("|").str[0]

/home/juan/Desktop/juan/bio/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [232]:
df_mirna_res_mites_for_merge = df_mirna_res_mites[['Name','#Locus','MajorRNA','MITE_prod']]

In [233]:
df_merge = pd.merge(df_mites,df_mirna_res_mites,left_on='Query', right_on='Name')

In [234]:
df_new = df_merge[['SiteID','DegradomeCategory','Name','#Locus','MajorRNA','MITE','MITE_prod']]

In [235]:
df_merge.head(2)

,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,...,22,23,24,seqname,position,start,end,MITE_prod,mirbase,annotation
0,TraesCS1A02G116200.1:3849,mrcv_all_Cluster_12767,TraesCS1A02G116200.1,3838,3858,3849,-37.0,-25.9,0.7,12.5,...,3,5,13,3A,746087222-746087306,746087222,746087306,MITE_660,oan-miR-1400-3p,"exon, three_prime_UTR"
1,TraesCS1A02G116200.2:3785,mrcv_all_Cluster_12767,TraesCS1A02G116200.2,3774,3794,3785,-37.0,-25.9,0.7,12.5,...,3,5,13,3A,746087222-746087306,746087222,746087306,MITE_660,oan-miR-1400-3p,"exon, three_prime_UTR"


In [236]:
df_merge['equal_mite'] = 0
for k,v in df_merge.iterrows():
    prods = v.MITE_prod.split(',')
    target = v.MITE.split(',')
    prods = set(filter(None, prods))
    target = set(filter(None, target))
    inter = target.intersection(prods)
    if inter:
        print(v.SiteID, v.Name, inter)
        df_merge.at[k, 'equal_mite'] = 1

TraesCS2D02G327200.1:2349 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS3B02G102200.1:1506 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS3B02G231700.2:1224 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS5B02G480100.1:1530 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS5D02G261800.1:1451 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS1D02G263500.1:890 mrcv_mites_MITE_T_120271 {'DTT_Taes_SBB_42j2-2'}
TraesCS5B02G444200.1:3882 mrcv_mites_MITE_T_100589 {'DTT_Tdur_Icarus_103H9-1', 'MITE_376', 'MITE_975', 'MITE_698'}
TraesCS1B02G366400.1:1967 mrcv_mites_MITE_T_80536 {'MITE_1745', 'MITE_577'}
TraesCS1B02G434600.1:501 mrcv_mites_MITE_T_80536 {'MITE_1745'}
TraesCS1D02G010200.1:2957 mrcv_mites_MITE_T_80536 {'DTT_Taes_Icarus_BJ239005-1'}
TraesCS2A02G281000.2:2638 mrcv_mites_MITE_T_80536 {'MITE_1745'}
TraesCS2A02G281000.3:2626 mrcv_mites_MITE_T_80536 {'MITE_1745'}
TraesCS2D02G229300.1:3246 mrcv_mites_MITE_T_80536 {'DTT_Osat_Pan_AF277239-1', 'MITE_1745', 'DTT_Hvul_Pan_M801L24-1', 'DTT_Taes_Pan_42j2-6', 'MITE_577', 'MITE_

In [237]:
df_merge[df_merge.Name=='sun_all_Cluster_8835']

,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,...,23,24,seqname,position,start,end,MITE_prod,mirbase,annotation,equal_mite
39,TraesCS1A02G138200.2:1948,sun_all_Cluster_8835,TraesCS1A02G138200.2,1936,1956,1948,-32.4,-23.2,0.716049,6.0,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,0
40,TraesCS2D02G327200.1:2349,sun_all_Cluster_8835,TraesCS2D02G327200.1,2337,2357,2349,-32.4,-24.5,0.756173,6.0,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,1
41,TraesCS3B02G102200.1:1506,sun_all_Cluster_8835,TraesCS3B02G102200.1,1494,1514,1506,-32.4,-24.5,0.756173,6.0,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,1
42,TraesCS3B02G231700.2:1224,sun_all_Cluster_8835,TraesCS3B02G231700.2,1212,1232,1224,-32.4,-25.2,0.777778,5.0,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,1
43,TraesCS3D02G116500.1:1764,sun_all_Cluster_8835,TraesCS3D02G116500.1,1754,1774,1764,-32.4,-22.7,0.700617,6.5,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,0
44,TraesCS4A02G331600.1:929,sun_all_Cluster_8835,TraesCS4A02G331600.1,917,937,929,-32.4,-22.5,0.694444,7.0,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,0
45,TraesCS4B02G313900.1:2810,sun_all_Cluster_8835,TraesCS4B02G313900.1,2798,2818,2810,-32.4,-21.4,0.660494,6.0,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,0
46,TraesCS5B02G218100.1:1564,sun_all_Cluster_8835,TraesCS5B02G218100.1,1553,1573,1564,-32.4,-26.1,0.805556,3.5,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,0
47,TraesCS5B02G480100.1:1530,sun_all_Cluster_8835,TraesCS5B02G480100.1,1518,1538,1530,-32.4,-24.5,0.756173,6.0,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,1
48,TraesCS5D02G261800.1:1451,sun_all_Cluster_8835,TraesCS5D02G261800.1,1439,1459,1451,-32.4,-24.5,0.756173,6.0,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,1


In [238]:
total_ele = len(df_merge.index)
equal_mite = len(df_merge[df_merge.equal_mite==1].index)
print(equal_mite * 100 /total_ele)

21.428571428571427


In [239]:
df_merge.to_csv(path_res_final, index=None)
path_res_final

'/home/juan/Desktop/juan/bio/mirna_mite/data/res/res_deg.csv'